<p style="text-align: center;font-size: 40pt">Use case 3 - Shoreline Monitoring</p>

# Overview 

Requirements
- [Error minimization](4-lesson_error_minimization.ipynb)

Objectives of this lesson:
- give an example of application for point cloud registration

# Introduction
In order to support environmental monitoring of freshwater bodies, an autonomous surface vessel named Lizhbeth was developed at the ASL in collaboration with the Limnological Station of the University of Zurich.
Although the vessel was initially developed to deploy biologic sensors in water (see [\[Hitz et al., 2012](https://ieeexplore.ieee.org/document/6153016), [2014\]](https://link.springer.com/chapter/10.1007%2F978-3-319-23778-7_49) for details on this application), a 3D laser was installed on its top to complement the analysis of the ecosystem with geological information.
For example, 3D mapping of the shoreline could help to determine the volume of organic material (leaves) falling in a lake, accurately identify inflows of water, quantify coastal erosion, etc.
The observation of coastal erosion using rangefinder laser is already an active field in geology [[Mitasova et al., 2009]](https://meridian.allenpress.com/jcr/article-abstract/25/2%20(252)/507/28283), relying mainly on airborne surveys.
However, this survey method can not provide a good viewpoint of a cliff, and the deployment costs are quite high.
The use of a boat as carrier is comparatively a low-cost method that can give better vantage points in certain situations. 
Beyond the geological applications, localization on the shore with centimeters precision can increase the autonomy of the system by allowing it to navigate to the sampling point from a parked position in a confined area, such as a boat house, or from its docking recharge station.
One of the requirement of such application is to have long range measurements, given that shores imply shallow water, which poses a limit on how close the boat can be without touching the bottom.
The type of outdoor environments expected when surveying a water body can vary from structured to unstructured, depending on the intensity of the recreational use by the local people.
For example scanned elements can be dams, bridges, houses, beaches, rocky shores, sparse to dense vegetation, etc.
Except for other boats, most of the environment is expected to be static with potential for seasonal changes (global motion) monitoring.

# Platform
The platform was deployed several times in Lake Zurich ([Figure 3.13](#robotLizhbeth)) and once in the alpine Lake Cadagno, both located in Switzerland.
It had a volume of 6.75 m$^3$ and weighted approximately 120 kg.
The motion of the robot was ensured by two electrical propellers positioned in the custom-built hulls of the catamaran.
This gave differential drive motion capability to the platform, allowing it to turn on spot.
The typical velocity of the robot is 0.7 m/s when surveying away from the shore.
The main sensor used for 3D reconstruction was a Velodyne HDL-32E, which produced in average 45,000 points.
The point clouds were recorded at 1.6 Hz.
A single-beam underwater sonar was used to produce bathymetric maps.
The localization sensors included an IMU, a magnetic compass and a GPS.
The GPS was mainly used for offshore navigation because its precision of 5 m made it dangerous for nearshore navigation.
The odometry can hardly be computed based on the motor inputs because of the high inertia of the boat in water, and the unknown wind-driven surface currents.
Among the main sources of localization perturbations are the waves that may change rapidly the platform orientation, which can be evaluated by the gravity vector measured by the IMU.
The smooth motion of the platform rendered difficult to extract reliable translation information without adding any registration algorithms.
Nevertheless, a predictive model implying smooth 2D translations on the $xy$-plane can be used to pre-align scans [[Hitz et al.,
2014]](https://link.springer.com/chapter/10.1007%2F978-3-319-23778-7_49).
<p id="robotLizhbeth" style="text-align: center;">
    <img src="images/robot_lizhbeth.jpg" width="50%"/> <br/>
    <b>Figure 3.13:</b> The autonomous surface vessel, named Lizhbeth, during one of its survey environment: pre-alpine Lake Zurich, Switzerland. 
	The sensor was not installed at the time of the photograph.
</p>

# Solution
While keeping the constraints of Lizhbeth in mind, we ran preliminary mapping experiments using the Velodyne installed on a small watercraft (7 m long).
The substitute boat was a monohull and thus, was considered less stable on water than Lizhbeth, which is a two-hull vessel.
As no external sensors were available, the full solution was tuned to not rely on any pre-alignment of the scans.
The input filters applied ensured that the watercraft was removed from the scans, and a fast random subsampling reduced the number of points to ensure registration at every 0.6 s.
The watercraft recorded scans while sailing, and its movements depended on water motions. Since waves induced fast changes in the watercraft's orientation, the matching of the scans needed to be fast enough to keep the error on the initial orientation small.
The size of the survey area was expected to be large, so a low density of points was forced.
When the laser hits the water it is usually not reflected back to the sensor, as opposed to solid ground.
Unfortunately, some waves can be detected by the laser because of their variable surface orientations.
To reduce wave-reflectance effect, we applied a strict shadow point filter that removes 3D points that display an angle difference larger than 17$^\circ$ between surface normals and the direction of observation.
The complete list of modules used with their main parameters are listed in [Table 3.5](#icpConfigAppLizhbeth).
<p id="icpConfigAppLizhbeth" style="text-align: center;margin-left: 15%;margin-right: 15%;">
    <b>Table 3.5:</b> Configuration of the ICP chain for the Lizhbeth mapping applications.
    The definition of the column <em>Step</em> follows previous lessons. 
    The names used in the column <em>Module</em> refer to specific implementation documented in the open source library <tt>libpointmatcher</tt>.
</p>

|    *Step*   |            *Module*           | *Description* |
|:-----------:|:-----------------------------:|:-------------:|
| $DF_{read}$ |      <tt>BoundingBox</tt>     | Remove points in a box of 7$\times$7$\times$2 m to avoid self-scanning. |
|             |    <tt>RandomSampling</tt>    | Keep randomly 20 % of the points. |
|             |     <tt>SurfaceNormal</tt>    | Compute normal and density with  20 NN and an approximation factor $\epsilon=3.16$. |
|             | <tt>ObservationDirection</tt> | Add vector pointing toward the origin of the sensor. |
|             |     <tt>OrientNormals</tt>    | Orient surface normals toward the observation direction. |
|             |      <tt>MaxDensity</tt>      | Subsample to keep point with density of 50 pts/m$^3$. |
|             |        <tt>Shadow</tt>        | Remove points with angle between surface normals and observation direction larger than 17$^\circ$. |
|  $DF_{ref}$ |        <tt>MaxDist</tt>       | Keep points within a radius of 70 m from the last sensor pose. |
|             |     <tt>SurfaceNormal</tt>    | Compute normal and density with 20 NN and an approximation factor $\epsilon=3.16$. |
|             |      <tt>MaxDensity</tt>      | Subsample to keep point with density of 50 pts/m$^3$. |
|             |     <tt>MaxPointCount</tt>    | Subsample 70 % if there is more than 600,000 points. |
|      MF     |        <tt>KDTree</tt>        | Use an approximate kD-tree with a maximum matching distance of 5.0 m and an approximation factor of $\epsilon=3.16$. |
|      OF     |      <tt>TrimmedDist</tt>     | Keep 90 % closest paired points. |
|             |     <tt>SurfaceNormal</tt>    | Remove paired points with normals angle larger than 90$^\circ$. |
|      EM     |     <tt>PointToPlane</tt>     | Objective function using point-to-plane error. |
|      TC     |     <tt>Differential</tt>     | Stop after a minimum error below 0.01 m and 0.001 rad. |
|             |        <tt>Counter</tt>       | Stop after the iteration count reached 40. |
|             |         <tt>Bound</tt>        | Stop if transformation increases beyond 5.0 m and 0.8 rad. |
<p style="text-align: center;margin-left: 15%;margin-right: 15%;font-size:10px;">
    <em>Legend</em>: $DF_{read}$ = Data Filters for readings, $DF_{ref}$ = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
</p>

# Results
The experiment executed with the watercraft was recorded on Lake Zurich, in front of the Limnological Station, the typical operating area of Lizhbeth.
The boat started away from the shore moving towards a harbor where multiple boats were parked side by side.
This starting position is located on the lower left corner of [Figure 3.14](#resultLizhbeth).
The boat first passed between the harbor and boats anchored on buoys and turned right to continue between the anchored boats and the shore.
The boat sailed parallel to the shore up to a boat house situated within an artificial small canal leading to the entrance of the boat house.
On the reconstructed environment, one can notice the noise around the anchored boats that was caused by their movements during the experiment, especially around the white one, at the bottom left corner.
Also seen in that corner are the noisy light gray points that were generated by the reflection of the laser on the waves.
The final map covered an area of 280 by 130 m without displaying any major defects.
<p id="resultLizhbeth" style="text-align: center;">
    <img src="images/kilchberg_combined.jpg" width="60%"/>
    <img src="images/kilchberg_overview_text.png" width="60%"/> <br/>
    <b>Figure 3.14:</b> Reconstruction of the shoreline from a boat.
    <em>Top</em>: Overlay of the 3D map with an aerial view.
    <em>Bottom</em>: Top view of the 3D map with point colors based on elevation, light gray being low and dark blue being high.
    The orthogonal projection of the aerial image was provided by the Bundesamt für Landestopografie swisstopo (Art. 30 GeoIV): 5704 000 000
</p>

The final solution must take into consideration that elements located offshore can have multiple possible positions.
By keeping the global map updated at every uses, those multiple positions will be retained in the map, thus reducing the chances of large drifts in cases where, for example, only the boats on the buoys were scanned.

# Conclusion
You should do the following activities to enhance your understanding of the concepts viewed in this lesson:
- modify the markdown by adding your own notes using `> my notes`; and
- complete the tables [Symbol definitions](#Symbol-definitions) and [Glossary](#Glossary) and add your own definitions.

Parallel lessons:
- [Use case 1](5-lesson_use_case_1.ipynb)
- [Use case 2](5-lesson_use_case_2.ipynb)

Next lesson:
- [Registration challenges](6-lesson_challenges.ipynb)

## Symbol definitions

| Symbol             | Definition  |
|--------------------|-------------|
|                    |             |

## Glossary

| English   | Français   | Definition |
|-----------|------------|------------|
|           |            |            |